# Testing the Cluster data_generator.py File

This file is meant to allow me to test the code in data_generator.py (which is what is used to generate data in the cluster) by visualizing it. That file only runs the model, with minimal pygame code, and so it is not easily visualizable. Ideally, the code from that file should be copy pasted in a cell below and then run to generate some data, and then the data can be visualized using some code that is already written below. There is also some code (immediately below this paragraph) that uses a slightly older (but likely equivalent) data_generator.py version to run the simulation in real time and see what ants moving around looks like.

## Visualizing Not-up-to-Date (but Potentially Equivalent) data_generator.py Code

In [153]:
#!/usr/bin/env python

import pygame
from pygame.math import Vector2 as Vector
import numpy as np
import math
import random
from random import randint
import copy
import os
import time


# ## Some Independent Functions

def get_grid_neighbors(x,y,w,h):
    neighbors = []
    for x2 in range(x-1, x+2):
        for y2 in range(y-1, y+2):
            if (
                -1 < x <= w-1 and -1 < y <= h-1 
                and (x != x2 or y != y2) 
                and (0 <= x2 <= w-1) and (0 <= y2 <= h-1)
               ):
                neighbors.append((x2,y2))
    return neighbors

# Define a function to map pheromone potential to color
def potential_to_color(potential, vmin, vmax):
    # Normalize the potential value
    normalized_value = (potential - vmin) / (vmax - vmin)
    
    # Map the normalized value to a color between light yellow (255, 255, 204) and dark red (189, 0, 38)
    color = (
        int(255 - 66 * normalized_value),
        int(255 - 255 * normalized_value),
        int(204 - 166 * normalized_value))
    
    return color


# ## Ant Class


class ANT:
    def __init__(self, x, y, num, IANT_RADIUS, IMIN_SPEED, IMAX_SPEED, IK_P, ISPEED_ALPHA,
                 IP_FORAGING, IP_HOMING, IMIN_PH_SENS, ITHETA_STOCHASTICITY, IVMAX, IIPS):
        # ANT CONSTANTS
        self.ANT_RADIUS = IANT_RADIUS
        self.MIN_SPEED = IMIN_SPEED
        self.MAX_SPEED = IMAX_SPEED
        self.K_P = IK_P
        self.SPEED_ALPHA = ISPEED_ALPHA
        self.P_FORAGING = IP_FORAGING
        self.P_HOMING = 1 - IP_FORAGING
        self.MIN_PH_SENS = IMIN_PH_SENS
        self.THETA_STOCHASTICITY = ITHETA_STOCHASTICITY
        self.VMAX = IVMAX
        self.IPS = IIPS
        self.CELL_SIZE = 5
        
        self.num = num
        self.pos = Vector(x,y)
        self.theta = random.uniform(0,2*np.pi)
        self.speed = self.MIN_SPEED #random.uniform(self.MIN_SPEED,self.MAX_SPEED)
        if random.uniform(0,1) <= self.P_FORAGING:
            self.mode = "FORAGING"
        else:
            self.mode = "HOMING"
    
    def draw_ant(self,surface):
        x_coord = round(self.pos.x) * self.CELL_SIZE
        y_coord = round(self.pos.y) * self.CELL_SIZE
        ant_rect = pygame.Rect(x_coord, y_coord, self.ANT_RADIUS*self.CELL_SIZE, self.ANT_RADIUS*self.CELL_SIZE)
        pygame.draw.rect(surface,'black',ant_rect)
        
    def avoid_wall(self,w,h):
        if round(self.pos.x) <= (1 + self.ANT_RADIUS) or round(self.pos.x) >= (w - 1 - self.ANT_RADIUS)             or int(self.pos.y) <= (self.ANT_RADIUS + 1) or round(self.pos.y) >= (h - 1 - self.ANT_RADIUS):
            center = (round(random.uniform(0.1*w,0.9*w)),round(random.uniform(0.1*h,0.9*h)))
            direction = (center[0] - round(self.pos.x), center[1] - round(self.pos.y))
            self.theta = math.atan2(direction[1], direction[0])
    
    def avoid_collision(self, ants):
        xdot = self.speed/self.IPS * np.cos(self.theta)
        ydot = self.speed/self.IPS * np.sin(self.theta)

        new_x_coord = round(self.pos.x + xdot)
        new_y_coord = round(self.pos.y + ydot)
        for ant in ants:
            if abs(new_x_coord - round(ant.pos.x)) == 0 and abs(new_y_coord - round(ant.pos.y)) == 0:
                self.theta += random.uniform(-0.5*self.THETA_STOCHASTICITY,0.5*self.THETA_STOCHASTICITY)
                return False
        return True
    
    # Basic function to find gradient by looking for neighbor with highest concentration of pheromone.
    def find_gradient_direction_1(self,pheromone_field):
        w, h = len(pheromone_field[0]), len(pheromone_field)
        x, y = round(self.pos.x), round(self.pos.y)
        x, y = min(max(x,0),w), min(max(y,0),h)
        neighbors = get_grid_neighbors(x,y,w,h)
        maximum_pheromone = self.MIN_PH_SENS * self.VMAX #potential_field[location_x][location_y]
        max_pheromone_neighbors = [(x,y)]
        for n in neighbors:
            n_x, n_y = n
            if pheromone_field[n_x][n_y] > 1.1 * maximum_pheromone:
                maximum_pheromone = pheromone_field[n_x][n_y]
                max_pheromone_neighbors = [(n_x,n_y)]
            elif pheromone_field[n_x][n_y] > 0.9 * maximum_pheromone:
                max_pheromone_neighbors.append((n_x,n_y))
        max_pheromone_neighbor = random.choice(max_pheromone_neighbors)

        return max_pheromone_neighbor, maximum_pheromone
    
    def move_ant(self, pheromone_field, ants):
        w, h = len(pheromone_field[0]), len(pheromone_field)
        if self.mode == "FORAGING":
            self.theta += random.uniform(-self.THETA_STOCHASTICITY,self.THETA_STOCHASTICITY)
            self.avoid_wall(w,h)
            
            self.speed = max(self.speed, (self.MIN_SPEED + self.MAX_SPEED) / 2)
            
            xdot = self.speed/self.IPS * np.cos(self.theta)
            ydot = self.speed/self.IPS * np.sin(self.theta)
            
            move = self.avoid_collision(ants)
            if move:
                self.pos.x += xdot
                self.pos.y += ydot
        
        elif self.mode == "HOMING":
            gradient_direction, gradient_V = self.find_gradient_direction_1(pheromone_field)
            gradient = (gradient_direction[0] - round(self.pos.x), gradient_direction[1] - round(self.pos.y))
            if gradient[0] != 0:
                gradient_theta = math.atan2(gradient[1], gradient[0])
            elif gradient[1] == 0:
                gradient_theta = self.theta + random.uniform(-self.THETA_STOCHASTICITY,self.THETA_STOCHASTICITY)
            else:
                gradient_theta = (np.pi/2)*(gradient[1] - round(self.pos.y))

            thetadot = self.K_P*np.sin(gradient_theta - self.theta)
            self.theta += thetadot

            self.avoid_wall(w,h)
            
            # slow down or speed up
            bounded_gradient_V = min(max(0,gradient_V),self.VMAX)
            target_speed = self.MAX_SPEED * (1 - bounded_gradient_V / self.VMAX)
            self.speed = self.speed + self.SPEED_ALPHA * (target_speed - self.speed)
            self.speed = min(max(self.MIN_SPEED,self.speed),self.MAX_SPEED)
            
            xdot = self.speed/self.IPS * np.cos(self.theta)
            ydot = self.speed/self.IPS * np.sin(self.theta)
            
            move = self.avoid_collision(ants)
            if move:
                self.pos.x += xdot
                self.pos.y += ydot


# ## Simulation Class


class SIM:
    def __init__(self, DIMS, IIPS, NUM_ANTS, IIC,
                 ID, IE, IVMIN, IVMAX, IM, IP_DROP, IPHEROMONE_INTERVAL,
                 IANT_RADIUS, IMIN_SPEED, IMAX_SPEED, IK_P, ISPEED_ALPHA,
                 IP_FORAGING, IP_HOMING, IMIN_PH_SENS, ITHETA_STOCHASTICITY):
        self.width, self.height = DIMS
        
        # PHEROMONE CONSTANTS
        self.IC = IIC
        self.D = ID #0.2
        self.E = IE #0.001
        self.VMIN = IVMIN
        self.VMAX = IVMAX
        self.M = IM
        self.P_DROP = IP_DROP
        self.PHEROMONE_INTERVAL = IPHEROMONE_INTERVAL
        self.IPS = IIPS
        self.CELL_SIZE = 5
        
        self.pheromone_field = [[0 for i in range(self.width+1)] for j in range(self.height+1)]
        
        # ANTS
        
        a_per_line = math.floor(math.sqrt(NUM_ANTS))
        a_spacing = (self.IC == "c")*1 + (not self.IC == "c")*math.floor(self.width / (a_per_line+1))
        X_OFFSET = 0 + (self.IC == "c")*round(self.width/2 - a_spacing*(a_per_line/2))
        Y_OFFSET = 0 + (self.IC == "c")*round(self.height/2 - a_spacing*(a_per_line/2))
        self.ants = [ANT(x * a_spacing + X_OFFSET, y * a_spacing + Y_OFFSET, y + (x-1)*a_per_line, 
                         IANT_RADIUS, IMIN_SPEED, IMAX_SPEED, IK_P, ISPEED_ALPHA,
                         IP_FORAGING, 1 - IP_FORAGING, IMIN_PH_SENS, 
                         ITHETA_STOCHASTICITY, IVMAX, IIPS) for x in range(1,a_per_line+1) for y in range(1,a_per_line+1)]
        
        # If ants start cluster, add pheromone around their initial conditions.
        if self.IC == "c":
            for i in range(X_OFFSET+1,X_OFFSET+a_per_line*a_spacing+1,1):
                for j in range(Y_OFFSET+1,Y_OFFSET+a_per_line*a_spacing+1,1):
                    self.pheromone_field[i][j] += self.VMAX
                
        # TIMER
        self.transpired_intervals = 0
    
    def update(self):
        self.transpired_intervals += 1
        self.diffuse_pheromone()
        self.drop_pheromone()
        for ant in self.ants:
            ant.move_ant(self.pheromone_field, self.ants)
    
    def draw_elements(self,surface,display):
        # fill the screen with white
        surface.fill((255, 255, 255))
        
#         # draw pheromone field
#         self.draw_pheromone(surface)
        
        # draw ants
        for ant in self.ants:
            ant.draw_ant(surface)
        
        pygame.display.update()
    
    def draw_pheromone(self,surface):
        p_width = len(self.pheromone_field[0])
        p_height = len(self.pheromone_field)
        
        for x in range(p_width):
            for y in range(p_height):
                concentration = self.pheromone_field[x][y]
                bounded_concentration = min(max(concentration,self.VMIN),self.VMAX)
                color = potential_to_color(bounded_concentration, self.VMIN, self.VMAX)
                
                x_coord, y_coord = int(x * self.CELL_SIZE), int(y * self.CELL_SIZE)
                # Draw a rectangle at the current point with the calculated color
                p_rect = pygame.Rect(x_coord, y_coord, self.CELL_SIZE, self.CELL_SIZE)
                pygame.draw.rect(surface, color, p_rect)
    
    def drop_pheromone(self):
        for ant in self.ants:
            if random.uniform(0,1) < self.P_DROP/self.IPS:
                # Determine ant coordinates.
                drop_x, drop_y = round(ant.pos.x), round(ant.pos.y)
                # Bound coordinates to within simulation dimensions.
                drop_x, drop_y = max(min(drop_x,self.width-1),1), max(min(drop_y,self.height-1),1)
                # Add drop.
                self.pheromone_field[drop_x][drop_y] = self.VMAX * self.M
    
    def diffuse_pheromone(self):
        p_width = len(self.pheromone_field[0])
        p_height = len(self.pheromone_field)
    
        old_field = copy.deepcopy(self.pheromone_field)
        for x in range(1,p_width-1):
            for y in range(1,p_height-1):
                square_concentration = old_field[x][y]
                neighbors = get_grid_neighbors(x,y,p_width,p_height)
                total_pheromone = square_concentration
                for n in neighbors:
                    total_pheromone += old_field[n[0]][n[1]]
                ave_pheromone = total_pheromone / (len(neighbors) + 1)
                next_concentration = (1 - self.E/self.IPS) * (square_concentration + (self.D/self.IPS * (ave_pheromone - square_concentration)))
                self.pheromone_field[x][y] = next_concentration


# ## Function to Run Simulation without Visualization


def run_simulation(IDURATION, IIPS, IWIDTH, IHEIGHT, ICELL_SIZE, IN_ANTS, IIC,
                   ID, IE, IVMIN, IVMAX, IM, IP_DROP, IPHEROMONE_INTERVAL, 
                   IANT_RADIUS, IMIN_SPEED, IMAX_SPEED, IK_P, ISPEED_ALPHA, 
                   IP_FORAGING, IP_HOMING, IMIN_PH_SENS, ITHETA_STOCHASTICITY):
    # SIMULATION CONSTANTS
    DURATION = IDURATION
    IPS = IIPS
    
    # User-set parameters.
    OUTPUT_DIR = "ostwald_frames4"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    display = False
    record = True
    save_ant_locations = False
    if save_ant_locations: 
        ant_locations = []

    # simulation and visualization dimensions
    SIM_WIDTH, SIM_HEIGHT = IWIDTH, IHEIGHT
    CELL_SIZE = ICELL_SIZE
    VIS_WIDTH, VIS_HEIGHT = CELL_SIZE*SIM_WIDTH, CELL_SIZE*SIM_HEIGHT

    NUM_ANTS = IN_ANTS

    # initialize simulation
    simulation = SIM((SIM_WIDTH,SIM_HEIGHT), IPS, NUM_ANTS, IIC,
                     ID, IE, IVMIN, IVMAX, IM, IP_DROP, IPHEROMONE_INTERVAL,
                     IANT_RADIUS, IMIN_SPEED, IMAX_SPEED, IK_P, ISPEED_ALPHA,
                     IP_FORAGING, IP_HOMING, IMIN_PH_SENS, ITHETA_STOCHASTICITY)

    # append initial conditions to ant locations
    if save_ant_locations: 
        current_ant_locations = []
        for ant in simulation.ants:
            current_ant_locations.append((ant.pos.x,ant.pos.y))
        ant_locations.append(current_ant_locations)

    # initialize visualization screen and simulation surface
    pygame.init()
    screen = pygame.display.set_mode((VIS_WIDTH,VIS_HEIGHT))

    # loop to display simulation
    running = True
    while running and simulation.transpired_intervals < DURATION*IPS:
        for event in pygame.event.get():
            # Click SPACE to end simulation.
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                running = False
            # Click on the screen to add pheromone.
            elif event.type == pygame.MOUSEBUTTONDOWN:
                # Check if the left mouse button was clicked
                if event.button == 1:
                    mouse_x, mouse_y = event.pos
                    drop_x, drop_y = round(mouse_x / CELL_SIZE), round(mouse_y / CELL_SIZE)
                    drop_x, drop_y = max(min(drop_x,SIM_WIDTH-1),1), max(min(drop_y,SIM_HEIGHT-1),1)
#                     simulation.pheromone_field[drop_x][drop_y] += VMAX*M
            # Click right arrow to start/stop recording simulation.
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_RIGHT:
                record = bool( (record + 1) % 2 )

        simulation.update()
        if display and simulation.transpired_intervals % 10 == 0:
            simulation.draw_elements(screen, display)
            
        if record and simulation.transpired_intervals % 1 == 0:
            simulation.draw_elements(screen, display)
            frame_filename = os.path.join(OUTPUT_DIR, f"frame_{simulation.transpired_intervals:04d}.png")
            pygame.image.save(screen, frame_filename)

        if save_ant_locations:
            current_ant_locations = []
            for ant in simulation.ants:
                current_ant_locations.append((ant.pos.x,ant.pos.y))
            ant_locations.append(current_ant_locations)
            
#         time.sleep(1/IPS)
    
#     return ant_locations


# # Parameter Analysis Data Collection

# SIMULATION CONSTANTS
iduration = 1000
iips = 5
iwidth = 100
iheight = 100
icell_size = 5
in_ants = 50
iic = "c"
# PHEROMONE CONSTANTS
iD = [0.2] #0.6
iE = [1] #0.4
iVMIN = 0
iVMAX = 100
iM = 8
iP_DROP = 0.3
iPHEROMONE_INTERVAL = 1
# ANT CONSTANTS
iANT_RADIUS = 1
iANT_LENGTH = 2
iMIN_SPEED = 4 #3
iMAX_SPEED = 20 #20
iK_P = 0.75
iK_S = 0.5
iP_FORAGING = 0.0
iP_HOMING = 1 - iP_FORAGING
iMIN_PH_SENS = 0.11
iTHETA_STOCHASTICITY = np.pi/16

reps = 1
trials = {}
print("Completed")
for d in iD:
    print("D:", d)
    p2trials = {}
    for e in iE:
        print("\tE:", e)
        p2reps = {}
        for i in range(reps):
            ant_trajectories = run_simulation(iduration, iips, iwidth, iheight, icell_size, in_ants, iic,
                                              d, e, 
                                              iVMIN, iVMAX, iM, iP_DROP, iPHEROMONE_INTERVAL, 
                                              iANT_RADIUS, iMIN_SPEED, iMAX_SPEED, iK_P, iK_S, 
                                              iP_FORAGING, iP_HOMING, iMIN_PH_SENS, iTHETA_STOCHASTICITY)
            p2reps["r"+str(i+1)] = ant_trajectories
        p2trials[e] = p2reps
    trials[d] = p2trials

# with open(output, "w") as f:
#     f.write(trials)

Completed
D: 0.2
	E: 1


In [14]:
len(trials[0.1][0.01]['r1'])

KeyError: 0.1

## Testing Current data_generator.py Code

In [ ]:
# Copy current code from data_generator.py here.



## To Visualize Trajectories

Run code above, saving data along the way, and then use the code below to visualize the trajectories.

In [ ]:
# simulation and visualization dimensions
sim_w, sim_h = 100, 100
cell_size = 5
vis_w, vis_h = cell_size*sim_w, cell_size*sim_h

pygame.init()
screen = pygame.display.set_mode((vis_w,vis_h))
pygame.display.set_caption("Simulation Trajectories")

# fill the screen with white
screen.fill((255, 255, 255))

### ###
### Plug in data that one wants visualized below.
### ###
ant_locations_transposed = [[(x * cell_size, y * cell_size) for x, y in row] for row in zip(*test_data)] #trials[0.1][0.001]['r1']

for ant in ant_locations_transposed:
    pygame.draw.lines(screen, (random.uniform(0,200),random.uniform(0,200),random.uniform(0,200)), False, ant)

pygame.display.update()